# 10 - Argon

**Overview** 

This notebook guides you through a molecular dynamics (MD) simulations of Argon atoms, looking at the effect of temperature on its structure and dynamics.

In [ ]:
# @title Modules Setup { display-mode: "form" }
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [ ]:
# @title Utilities { display-mode: "form" }
# -------------------------------
# Utilities: PBC + LJ definitions
# -------------------------------

def minimum_image(dr, cell):
    """Apply minimum image convention to displacement vectors."""
    return dr - np.rint(dr / cell) * cell

def lj_potential(r, epsilon, sigma):
    """Bare Lennard-Jones potential (no shifting). r>0."""
    sr6 = (sigma / r)**6
    return 4 * epsilon * (sr6**2 - sr6)

def lj_force_scalar(r, epsilon, sigma):
    """Magnitude/r of the pair force (so that F_vec = f(r) * r_hat)."""
    sr2 = (sigma / r)**2
    sr6 = sr2**3
    return 24 * epsilon * (2 * sr6**2 - sr6) / (r**2)

def shifted_lj(r, epsilon, sigma, rcut):
    """Energy and force with potential and force shifted to 0 at rcut."""
    U = lj_potential(r, epsilon, sigma)
    f_over_r = lj_force_scalar(r, epsilon, sigma)

    Uc = lj_potential(rcut, epsilon, sigma)
    fc_over_r = lj_force_scalar(rcut, epsilon, sigma)

    U_shifted = U - Uc - (r - rcut) * (fc_over_r * rcut)
    f_over_r_shifted = f_over_r - fc_over_r

    return U_shifted, f_over_r_shifted

# -------------------------------
# MD Simulator with thermostat
# -------------------------------

class MD2DLJ:
    def __init__(
        self,
        n_side=10,
        cell=np.array([30.0, 30.0]),
        mass=1.0,
        epsilon=1.6e-3,
        sigma=6.0,
        dt=0.01,
        rcut=None,
        seed=42,
        init_speed=1e-3,
        zero_net_momentum=True,
        # Thermostat options
        target_temperature=None,      # None = NVE, no thermostat
        thermostat='berendsen',       # 'berendsen' or 'velocity_rescale'
        thermostat_tau=1.0,           # relaxation time (for Berendsen)
        thermostat_interval=1,        # apply every N substeps
    ):
        self.n_side = int(n_side)
        self.n = self.n_side**2
        self.cell = np.array(cell, dtype=float)
        self.mass = float(mass)
        self.epsilon = float(epsilon)
        self.sigma = float(sigma)
        self.dt = float(dt)
        self.rcut = float(rcut) if rcut is not None else None

        # Thermostat parameters
        self.kB = 1.0
        self.dim = 2
        self.n_dof = self.dim * self.n
        self.target_temperature = target_temperature
        self.thermostat = thermostat
        self.thermostat_tau = thermostat_tau
        self.thermostat_interval = thermostat_interval
        self.step_count = 0

        # --- RDF setup (2D) ---
        self.rdf_nbins = 100              # number of histogram bins
        self.rdf_r_max = 0.5 * min(self.cell)  # only meaningful up to half box length
        self.rdf_counts = np.zeros(self.rdf_nbins, dtype=float)
        self.rdf_samples = 0              # number of configurations accumulated

        rng = np.random.default_rng(seed)

        # Grid initial positions
        xs, ys = np.meshgrid(
            np.linspace(0, self.cell[0] * (1 - 1/self.n_side), self.n_side),
            np.linspace(0, self.cell[1] * (1 - 1/self.n_side), self.n_side),
            indexing="ij"
        )
        self.r = np.vstack([xs.ravel(), ys.ravel()]).T

        # Small random velocities
        self.v = rng.uniform(-1, 1, size=(self.n, 2)) * init_speed

        if zero_net_momentum:
            v_cm = self.v.mean(axis=0, keepdims=True)
            self.v = self.v - v_cm

        # Initial forces and time
        self.F = self.compute_forces(self.r)
        self.t = 0.0

        # Histories
        self.kinetic_history = []
        self.potential_history = []
        self.total_history = []
        self.time_history = []
        self.temperature_history = []

        self._record()

    # -------------
    # Core routines
    # -------------

    def compute_forces(self, r):
        """Vectorized pair forces with PBC and optional cutoff + shifting."""
        dr = r[:, None, :] - r[None, :, :]
        dr = minimum_image(dr, self.cell)
        rij = np.linalg.norm(dr, axis=-1)

        np.fill_diagonal(rij, np.inf)

        if self.rcut is not None:
            mask = rij < self.rcut
            f_over_r = np.zeros_like(rij)
            _, f_over_r_cut = shifted_lj(rij[mask], self.epsilon, self.sigma, self.rcut)
            f_over_r[mask] = f_over_r_cut
        else:
            mask = np.isfinite(rij)
            f_over_r = np.zeros_like(rij)
            f_over_r[mask] = lj_force_scalar(rij[mask], self.epsilon, self.sigma)

        F = np.sum((f_over_r[..., None]) * dr, axis=1)
        return F

    def potential_energy(self, r):
        dr = r[:, None, :] - r[None, :, :]
        dr = minimum_image(dr, self.cell)
        rij = np.linalg.norm(dr, axis=-1)
        iu, ju = np.triu_indices(self.n, k=1)

        if self.rcut is not None:
            mask = rij[iu, ju] < self.rcut
            U = np.zeros_like(rij[iu, ju])
            U_shift, _ = shifted_lj(rij[iu, ju][mask], self.epsilon, self.sigma, self.rcut)
            U[mask] = U_shift
            return np.sum(U)
        else:
            return np.sum(lj_potential(rij[iu, ju], self.epsilon, self.sigma))

    def kinetic_energy(self):
        return 0.5 * self.mass * np.sum(self.v**2)

    def temperature(self):
        """Instantaneous temperature (kB=1)."""
        K = self.kinetic_energy()
        return 2.0 * K / (self.n_dof * self.kB)

    def apply_pbc(self):
        self.r = self.r - np.floor(self.r / self.cell) * self.cell

    def apply_thermostat(self):
        """Apply simple thermostat to velocities."""
        if self.target_temperature is None:
            return
        if self.thermostat is None:
            return

        T_curr = self.temperature()
        if T_curr <= 0:
            return

        if self.thermostat == 'velocity_rescale':
            # Instant velocity rescaling to hit target exactly
            lam = np.sqrt(self.target_temperature / T_curr)
        elif self.thermostat == 'berendsen':
            # Berendsen weak coupling
            tau = self.thermostat_tau
            dt = self.dt
            lam = np.sqrt(1.0 + dt / tau * (self.target_temperature / T_curr - 1.0))
        else:
            return  # unknown thermostat

        self.v *= lam

    def accumulate_rdf(self):
        """
        Accumulate radial distribution function g(r) from current positions.
        2D, with periodic boundary conditions.
        """
        # Pairwise displacements
        dr = self.r[:, None, :] - self.r[None, :, :]
        dr = minimum_image(dr, self.cell)
        rij = np.linalg.norm(dr, axis=-1)

        # Use only i < j to avoid double counting and self-distances
        iu, ju = np.triu_indices(self.n, k=1)
        dist = rij[iu, ju]

        # Histogram in [0, r_max]
        counts, edges = np.histogram(
            dist, bins=self.rdf_nbins, range=(0.0, self.rdf_r_max)
        )

        self.rdf_counts += counts
        self.rdf_samples += 1    

    def get_rdf(self):
        """
        Return (r_centers, g_r) for the accumulated RDF.
        2D normalization: g(r) = n_k / [ 2π r Δr ρ N ],
        where n_k is avg. number of pairs in the shell.
        """
        if self.rdf_samples == 0:
            raise RuntimeError("No RDF data accumulated. Call accumulate_rdf() during the run.")

        counts = self.rdf_counts / self.rdf_samples   # average over snapshots

        # Bin centers
        edges = np.linspace(0.0, self.rdf_r_max, self.rdf_nbins + 1)
        r_centers = 0.5 * (edges[1:] + edges[:-1])
        dr = edges[1] - edges[0]

        # 2D density
        A = self.cell[0] * self.cell[1]
        rho = self.n / A

        # Avoid division by zero at r = 0 (first bin center > 0 anyway if r_max>0)
        shell_areas = 2.0 * np.pi * r_centers * dr

        # g(r): counts per particle / ideal-gas counts per shell in 2D
        g_r = counts / (shell_areas * rho * self.n)

        return r_centers, g_r

    def step(self, n_substeps=1):
        """Advance the system by n_substeps of Velocity-Verlet."""
        dt = self.dt
        m = self.mass

        for _ in range(n_substeps):
            # r(t+dt)
            self.r = self.r + self.v * dt + 0.5 * (self.F / m) * dt**2
            self.apply_pbc()

            # F(t+dt)
            F_new = self.compute_forces(self.r)

            # v(t+dt)
            self.v = self.v + 0.5 * (self.F + F_new) * (dt / m)

            self.F = F_new
            self.t += dt
            self.step_count += 1

            # Thermostat (every thermostat_interval substeps)
            if (self.step_count % self.thermostat_interval) == 0:
                self.apply_thermostat()

    def run(self, steps=1000, substeps_per_frame=10, record_every=1):
        for s in range(steps):
            self.step(n_substeps=substeps_per_frame)
            if (s % record_every) == 0:
                self._record()

    def _record(self):
        ke = self.kinetic_energy()
        pe = self.potential_energy(self.r)
        self.kinetic_history.append(ke)
        self.potential_history.append(pe)
        self.total_history.append(ke + pe)
        self.time_history.append(self.t)
        self.temperature_history.append(self.temperature())
        self.accumulate_rdf()

    # -------------
    # Visualization
    # -------------

    def animate(self, frames=200, substeps_per_frame=10, dpi=120, tail=0):
        fig = plt.figure(figsize=(7, 9), dpi=dpi)
        gs = fig.add_gridspec(2, 1, height_ratios=[4, 1], hspace=0.3)

        ax0 = fig.add_subplot(gs[0, 0])
        ax1 = fig.add_subplot(gs[1, 0])

        ax0.set_xlim(0, self.cell[0])
        ax0.set_ylim(0, self.cell[1])
        ax0.set_aspect('equal', adjustable='box')
        ax0.set_title("2D Lennard–Jones MD (periodic)")

        scat = ax0.scatter(self.r[:, 0], self.r[:, 1], s=40)

        ln_total, = ax1.plot(self.time_history, self.total_history, label='Total E')
        ln_pot,   = ax1.plot(self.time_history, self.potential_history, label='Potential E')
        ln_kin,   = ax1.plot(self.time_history, self.kinetic_history, label='Kinetic E')
        ax1.set_xlabel("Time")
        ax1.set_ylabel("Energy")
        ax1.legend(loc="best")
        ax1.grid(True, alpha=0.3)

        def _advance():
            self.step(n_substeps=substeps_per_frame)
            self._record()

        def init():
            return scat, ln_total, ln_pot, ln_kin

        def update(frame):
            _advance()
            scat.set_offsets(self.r)

            if tail > 0:
                t = self.time_history[-tail:]
                tot = self.total_history[-tail:]
                pot = self.potential_history[-tail:]
                kin = self.kinetic_history[-tail:]
            else:
                t = self.time_history
                tot = self.total_history
                pot = self.potential_history
                kin = self.kinetic_history

            ln_total.set_data(t, tot)
            ln_pot.set_data(t, pot)
            ln_kin.set_data(t, kin)
            ax1.relim()
            ax1.autoscale_view()

            return scat, ln_total, ln_pot, ln_kin

        anim = animation.FuncAnimation(
            fig, update, init_func=init, frames=frames, interval=1, blit=False
        )
        plt.close(fig)
        return anim

# -------------------------------
# Helper: energy plotting
def plot_energies(t, sim, which="All"):
    plt.figure(figsize=(7,4), dpi=120)

    if which == "Potential":
        plt.plot(t, sim.potential_history, label="Potential E", linewidth=2)
        plt.ylabel("Potential energy")
    elif which == "Kinetic":
        plt.plot(t, sim.kinetic_history, label="Kinetic E", linewidth=2)
        plt.ylabel("Kinetic energy")
    elif which == "Total":
        plt.plot(t, sim.total_history, label="Total E", linewidth=2)
        plt.ylabel("Total energy")
    else:  # which == "All"
        plt.plot(t, sim.total_history, label="Total E", linewidth=2)
        plt.plot(t, sim.potential_history, label="Potential E", linewidth=2)
        plt.plot(t, sim.kinetic_history, label="Kinetic E", linewidth=2)
        plt.ylabel("Energy")

    plt.xlabel("Time")
    plt.title("Energies vs Time")
    plt.grid(True, alpha=0.3)
    plt.legend(loc="best")
    plt.tight_layout()
    plt.show()

# Helper: temperature plotting (if you have temperature_history)
def plot_temperature(t, sim):
    if not hasattr(sim, "temperature_history") or len(sim.temperature_history) == 0:
        print("No temperature history recorded.")
        return
    plt.figure(figsize=(7,4), dpi=120)
    plt.plot(t, sim.temperature_history, linewidth=2)
    plt.xlabel("Time")
    plt.ylabel("Temperature (reduced units)")
    plt.title("Temperature vs Time")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

# Helper: RDF plotting (using get_rdf from earlier)
def plot_rdf(sim):
    try:
        r, g_r = sim.get_rdf()
    except Exception as e:
        print("Cannot compute RDF:", e)
        return

    plt.figure(figsize=(7,4), dpi=120)
    plt.plot(r / sim.sigma, g_r, linewidth=2)
    plt.xlabel(r"$r/\sigma$")
    plt.ylabel(r"$g(r)$")
    plt.title("Radial distribution function $g(r)$")
    plt.grid(True, linestyle=":", alpha=0.6)
    plt.tight_layout()
    plt.show()
# -------------------------------

**Problem** 

Ideal gasses are a staple of physical chemistry and provide very intuitive laws to describe independent particles. However, even the most noble of atoms interact with dispersion and repulsion interactions and their physical properties will deviate from the ideal gas limit. Here we want to study a gas (or a liquid) or Ar atoms as a function of temperature. In order to simplify the visualization of the system, we will consider a two-dimensional simulation, which may be relevant for the modeling of confined particles. 

**Model**

We will model the interaction between noble element atoms using a Lennard-Jones potential of the form

$$U_{\mathrm{LJ}}(r_{ij}) =
4\varepsilon
\left[
\left( \frac{\sigma}{r_{ij}} \right)^{12}
-
\left( \frac{\sigma}{r_{ij}} \right)^{6}
\right]$$

This is a very typical pairwise non-bonded interaction term in classical force-fields. What this means is that this expression, which relies on two empirical parameters ($\varepsilon$ and $\sigma$), describes the interaction between two nuclei at any distance, without the need to characterize the whole electronic structure of the system. This energy expression can thus be used together with Classical Mechanics (Newton's laws) to describe the time evolution of the atoms. 

We will use Classical MD, with a Velocity Verlet integrator and some simplified thermostats, to sample the configurations of a system of Ar atoms and extract its properties. 

>What happens to the structure of the system as the temperature is lowered?



### Part 1: Equilibration

One of the key aspects of MD sampling is that it usually starts from a configuration of atoms that is far (often VERY far) from the equilibrium configurations for the desired thermodynamic conditions. While any configuration compatible with our thermodynamic ensenble variables should be relevant, its weight compared to the equilibrium configurations may be completely wrong if we only run the dynamics for a short amount of time. Typically we want to discard the initial part of our trajectory, a.k.a. the equilibration step. Note that in general we may have multiple equilibration steps and transient parts of the trajectory that we need to discard. 

**Questions**

Before you run any simulation, answer the following question(s):

1. What is the physical origin of the two terms that form the Lennard-Jones potential? Draw the LJ potential and the corresponding force.
2. What would be the potential and force for two non-interacting particles? What would be the potential and force for hard-spheres (i.e. two particles that only interact when they hit each other)? 

Run the simulation, change the parameters, and run the simulation again as many times as needed to answer the following question(s):

3. The first key parameter in MD simulations is the timestep. The best timestep is the largest possible that is consistent with energy conservation (no drifts in the total energy). To check that the total energy of the system is conserved, we need to run an NVE simulation. Adjust the timestep to make sure the total energy of the system is conserved.
4. The second key parameter in MD simulations (both for equilibration and for production runs) is the number of steps. For equilibration runs this means making sure that the main thermodynamic variables of the system have reached their equilibrium average and are only fluctuating around the average (no drifts). Some thermodynamic variables may take longer than others to reach equilibrium. Setup the dynamics in a canonical (NVT) ensemble by specifying a target temperature and a thermostat, and run the equilibration steps in chuncks as many times as needed to reach an equilibrium configuration. 
5. The number of steps required to equilibrate in the NVT ensemble will depend on the desired temperature and on the thermostat parameters. Typical thermostats have an associated time constant that gives an idea of how long it will take. Try to change the thermostat parameters and observe how they affect the equilibration.

In [ ]:
# @title Energy and Force Definitions and Visualization  { display-mode: "form" }
sigma = 3.4  # @param {type:"number"}
epsilon = 0.0016  # @param {type:"number"}
rcut = 8  # @param {type:"number"}

# --- Grid for plotting ---
r = np.linspace(0.85 * sigma, 3.0 * sigma, 500)  # avoid r→0 divergence

# Potentials
ulj = lj_potential(r, epsilon, sigma)
ulj_shifted, flj_shifted_over_r = shifted_lj(r, epsilon, sigma, rcut)

# Forces: lj_force_scalar returns F/r, so multiply by r to get radial force
flj_over_r = lj_force_scalar(r, epsilon, sigma)
F_lj = flj_over_r * r
F_shifted = flj_shifted_over_r * r

# Reduced-unit versions
U_lj_red = ulj / epsilon
U_lj_shifted_red = ulj_shifted / epsilon
F_lj_red = F_lj * sigma / epsilon
F_shifted_red = F_shifted * sigma / epsilon

# Location of LJ minimum
r_min = 2**(1/6) * sigma

# --- Plot setup: 2 panels (U and F) ---
fig, (axU, axF) = plt.subplots(
    2, 1, figsize=(8, 6), sharex=True,
    gridspec_kw={'hspace': 0.05}
)

# --- Potential panel ---
axU.plot(r / sigma, U_lj_red, label=r'$U_{\mathrm{LJ}}(r)$', linewidth=2)
axU.plot(r / sigma, U_lj_shifted_red, label=r'$U_{\mathrm{LJ}}^{\mathrm{shift}}(r)$',
         linewidth=2, linestyle='--')

axU.axhline(0.0, color='k', linewidth=0.8, alpha=0.7)
axU.axvline(r_min / sigma, color='gray', linewidth=1.0, linestyle=':',
            label=r'$r_\mathrm{min}$')
axU.axvline(rcut / sigma, color='red', linewidth=1.0, linestyle='--',
            label=r'$r_\mathrm{cut}$')

axU.set_ylabel(r'$U/\varepsilon$', fontsize=12)
axU.set_title('Lennard–Jones potential and force (Argon-like parameters)', fontsize=14)
axU.grid(True, linestyle=':', alpha=0.6)

# Merge legend entries to avoid duplicates
handles, labels = axU.get_legend_handles_labels()
axU.legend(handles, labels, fontsize=10, loc='best')

# --- Force panel ---
axF.plot(r / sigma, F_lj_red, label=r'$F_{\mathrm{LJ}}(r)$', linewidth=2)
axF.plot(r / sigma, F_shifted_red, label=r'$F_{\mathrm{LJ}}^{\mathrm{shift}}(r)$',
         linewidth=2, linestyle='--')

axF.axhline(0.0, color='k', linewidth=0.8, alpha=0.7)
axF.axvline(r_min / sigma, color='gray', linewidth=1.0, linestyle=':')
axF.axvline(rcut / sigma, color='red', linewidth=1.0, linestyle='--')

axF.set_xlabel(r'Reduced distance $r/\sigma$', fontsize=12)
axF.set_ylabel(r'$F\sigma/\varepsilon$', fontsize=12)
axF.grid(True, linestyle=':', alpha=0.6)
axF.legend(fontsize=10, loc='best')

plt.show()

In [ ]:
# @title Equilibration Parameters  { display-mode: "form" }
sigma = 3.4  
epsilon = 0.0016  
rcut = 2.5 * sigma  
mass = 1. 
dt = 0.02 # @param {type:"number"}
N = 100  # @param {type:"integer"} 
L = 70.0  # @param {type:"number"}
n_side = int(np.sqrt(N))  # particles per side of the grid
T = 0.001  # @param {type:"number"}
thermostat_type = 'berendsen'  # @param ["berendsen", "velocity_rescale", "none"]
thermostat_tau = 5.0  # @param {type:"number"}

sim_eq = MD2DLJ(
    n_side=n_side,
    cell=np.array([L, L]),
    mass=mass,
    epsilon=epsilon,
    sigma=sigma,
    dt=dt,
    rcut=rcut,
    seed=0,
    init_speed=1e-3,
    # Thermostat settings:
    target_temperature=T,      # choose a target T (in reduced units)
    thermostat=thermostat_type,      # or 'velocity_rescale'
    thermostat_tau=thermostat_tau,          # how strongly we couple to the bath
    thermostat_interval=1,       # apply every substep
)

In [ ]:
# @title Run and Visualize the Simulation  { display-mode: "form" }
animate = True  # @param {type:"boolean"}
nsteps = 400 # @param {type:"integer"}
steps_per_frame = 20 # @param {type:"integer"}
nrecord = 20 # @param {type:"integer"}
show = "All" # @param ["Potential", "Kinetic", "Total", "Temperature", "All"]

if animate:
    anim = sim_eq.animate(frames=nsteps, substeps_per_frame=steps_per_frame, dpi=120)
#    display(HTML(anim.to_html5_video()))
    display(HTML(anim.to_jshtml()))
else:
    # Run the dynamics and record energies / temperature / RDF
    sim_eq.run(steps=nsteps, substeps_per_frame=steps_per_frame, record_every=nrecord)

    t = np.array(sim_eq.time_history)
  
    # --- Dispatch based on 'show' selection ---
    if show in ["Potential", "Kinetic", "Total"]:
        # Show only one energy component
        plot_energies(t, sim_eq, which=show)

    elif show == "Temperature":
        # Only temperature
        plot_temperature(t, sim_eq)

    elif show == "All":
        # 1) All energies together
        plot_energies(t, sim_eq, which="All")
        # 2) Temperature
        plot_temperature(t, sim_eq)

### Part 2: Production

Once we have a good, fully equilibrated, starting configuration, we can perform a MD simulation to sample configurations in time. This is usually called a production run, and there is no limit on how long we run it. The advantage of MD simulations is that we can almost always restart them, if we need to add more statistics or refine our results. However, one of the main challenges with MD simulations is how to process the large amount of information generated.

**Questions**

Before you run any simulation, answer the following question(s):

1. For an ideal gas, what is the probability of finding a particle at a certain distance r from a given particle? What about a system of hard spheres?

Run the simulation, change the parameters, and run the simulation again as many times as needed to answer the following question(s):

2. For the given temperature equilibrated in the previous step, visualize the radial distribution function. How does it compare to the answers to the previous question? 
3. Change the temperature and volume of your system and recompute the radial distribution function. What is the effect of temperature on the structural features of the system?


In [ ]:
# @title Production Run  { display-mode: "form" }
nsteps = 2000 # @param {type:"integer"}
steps_per_frame = 20 # @param {type:"integer"}
nrecord = 10 # @param {type:"integer"}

sim_prod = MD2DLJ(
    n_side=sim_eq.n_side,
    cell=sim_eq.cell.copy(),
    epsilon=sim_eq.epsilon,
    sigma=sim_eq.sigma,
    mass=sim_eq.mass,
    dt=sim_eq.dt,
    rcut=sim_eq.rcut,
    target_temperature=None,   # NVE production
    thermostat=None,
)

# Copy microscopic state
sim_prod.r = sim_eq.r.copy()
sim_prod.v = sim_eq.v.copy()
sim_prod.F = sim_eq.F.copy()

# Reset time and histories
sim_prod.t = 0.0
sim_prod.kinetic_history = []
sim_prod.potential_history = []
sim_prod.total_history = []
sim_prod.time_history = []
if hasattr(sim_prod, "temperature_history"):
    sim_prod.temperature_history = []

if hasattr(sim_prod, "rdf_counts"):
    sim_prod.rdf_counts[:] = 0.0
    sim_prod.rdf_samples = 0

# Record initial production point
sim_prod._record()

# Run production
sim_prod.run(steps=nsteps, substeps_per_frame=steps_per_frame, record_every=nrecord)
t = np.array(sim_prod.time_history)

# 1) All energies together
plot_energies(t, sim_prod, which="All")
# 2) Temperature
plot_temperature(t, sim_prod)
# 3) RDF
plot_rdf(sim_prod)

**Homework Assignment**

For the last assignment, I would like you to do a literature search on simulations connected to your project. Find a few (minimum 3) publications that are connected to the system and simulations that you plan to study in your project. Report the types of simulations, system sizes and details, as well as the settings of the simulations in a text document. Comment on wether any relevant parameter is missing from the published works and if you would be able to reproduce some of the reported results with the tools used in this course. 